# Libraries

In [1]:
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt

# Input data

In [2]:
L = 1                             # Beam length in meters    
q = 100                           # load
nel = 1024                        # number of elements
nnos = nel + 1                    # number of nodes   
alldof = np.linspace(1, 1,2*nnos) # all degrees of freedom
kg = np.zeros((2*nnos,2*nnos))    # global stiffness matrix pre-allocation
I = 100                           # inertia
E = 210e9                         # young's modulus   
coord = np.zeros((nnos, 3))       # coordinate matrix pre-allocation
inci = np.zeros((nel, 6))         # incidence matrix pre-allocation
f = np.zeros((2*nnos, 1))         # external load vector

# Coordinate matrix

In [ ]:
for i in range(0, nnos):
    coord[i,0] = i + 1      # node number
    coord[i,1] = i*L/nel    # node position
    coord[i,2] = 0
l = coord[1,1] - coord[0,1] # element length

# Incidence matrix

In [ ]:
for i in range(0, nel):   
    inci[i,0] = i + 1          # element number
    inci[i,1] = i + 1          # first node
    inci[i,2] = i + 2          # second node
    inci[i,3] = coord[i,1]     # first node coordinate
    inci[i,4] = coord[i+1,1]   # second node coordinate
    if inci[i,4] <= 0.5:       # inertia of beam section
        inci[i,5] = 2*I
    else:
        inci[i,5] = I

# Boundary conditions

In [ ]:
#   bc=[node | degree of freedom | value]
#
#   Degree of freedom 1 --> y
#   Degree of freedom 2 --> oz

bc = np.array([[1,1,0],[1,2,0],[nnos,1,0]])

# Mask stiffness matrix

In [ ]:
mask = np.zeros((2*nnos,2*nnos))
for i in range(0, np.size(bc,0)):
    if bc[i,1] == 1:
        mask[2*bc[i,0]-2,2*bc[i,0]-2] = 1
    elif bc[i,1] == 2:
        mask[2*bc[i,0]-1,2*bc[i,0]-1] = 1
mask = ma.masked_equal(mask, 1)
mask = ma.mask_rowcols(mask)
mask = (mask==False)

# Mask load vector

In [ ]:
maskv = np.zeros(2*nnos)
for i in range(0, np.size(bc,0)):
    if bc[i,1] == 1:
        maskv[2*bc[i,0]-2] = 1
    elif bc[i,1] == 2:
        maskv[2*bc[i,0]-1] = 1
maskv = ma.masked_equal(maskv, 1)
maskv = (maskv==False)

#  Load vector

In [ ]:
#   F = [node | degree of freedom | value]
#
#   Degree of freedom 1 --> Fy
#   Degree of freedom 2 --> Mz
j = 1
k = 0
for i in range(0,nel):
    node1 = inci[i,1] # first node element
    node2 = inci[i,2] # second node element
    if coord[int(inci[i,1]),1] <= 0.5:
        loc = [int(2*node1-2), int(2*node1-1)]
        f[loc] = f[loc] - [[q*l/2],[q*l**2/12]]
    if coord[int(inci[i,1]),1] <= 0.5:
        loc = [int(2*node2-2), int(2*node2-1)]
        f[loc] = f[loc] - [[q*l/2],[q*l**2/12]]

# Global matrix assembly

In [ ]:
for i in range(nel):
    node1 = inci[i,1] # first node element
    node2 = inci[i,2] # second node element
    
    # local stiffness matrix
    inertia = inci[i,5]
    ke = E*inertia/l**3*np.array([[12, 6*l, -12, 6*l], [6*l, 4*l**2, -6*l, 2*l**2], [-12, -6*l, 12, -6*l],  [6*l, 2*l**2, -6*l, 4*l**2]])
    
    # localization vector
    loc = [2*node1-2,2*node1-1,2*node2-2,2*node2-1]
    
    # global stiffness matrix 
    kg[[[int(loc[0])],[int(loc[1])],[int(loc[2])], [int(loc[3])]], [int(loc[0]),int(loc[1]),int(loc[2]),int(loc[3])]] = kg[[[int(loc[0])],[int(loc[1])],[int(loc[2])], [int(loc[3])]], [int(loc[0]),int(loc[1]),int(loc[2]),int(loc[3])]] +  ke
    
kg_aux = kg[mask.data]
kg_aux = np.reshape(kg_aux, (2*nnos-np.size(bc,1), 2*nnos-np.size(bc,1)))
f_aux  = f[maskv.data]

# Displacement

In [ ]:
u = np.zeros((2*nnos, 1))
u[maskv.data] = np.linalg.solve(kg_aux, f_aux)
u = np.reshape(u,(nnos,2))
displacement = u[:,0]
rotation = u[:,1]

# Post-processing

In [ ]:
coord[:,2] = displacement
fig, ax = plt.subplots(figsize = (12, 7))

plt.plot(coord[:,1],coord[:,2])
plt.xlim([0,1])
plt.grid(b=True, which='major', color='k', linestyle='-')

fig.savefig('fem3_a.png',dpi=300)                              # save figure as png (dpi = number of pixels)